In [12]:
import os
import time

import dataflows as DF

from datapackage_pipelines_migdar.flows.dump_to_es import my_dump_to_es, BoostingMappingGenerator, collate
from datapackage_pipelines_migdar.flows.constants import REVISION


# kubectl port-forward to the migdar elasticsearch
os.environ["DATAFLOWS_ELASTICSEARCH"] = "localhost:9200"

# download from the pipelines pod data/ folder
data_source = '../data/publications_in_es/datapackage.json'

revision = REVISION
resource_name = 'publications'


def fix_rows(rows):
    for row in rows:
        row['revision'] = revision
        row['create_timestamp'] = time.time()


DF.Flow(
    DF.load(data_source),
    fix_rows,
    my_dump_to_es(
        indexes={
            'migdar__' + resource_name: [
                {
                    'resource-name': resource_name,
                    'revision': revision
                }
            ]
        },
        mapper_cls=BoostingMappingGenerator,
        index_settings={'index.mapping.coerce': True},
        elasticsearch_options=dict(timeout=60)
    ),
    collate(revision),
    my_dump_to_es(
        indexes={
            'migdar__docs': [
                {
                    'resource-name': resource_name,
                    'revision': revision
                }
            ]
        },
        mapper_cls=BoostingMappingGenerator,
        index_settings={'index.mapping.coerce': True},
        elasticsearch_options=dict(timeout=60)
    ),
).process()

ERROR:root:Failed to connect to database <Elasticsearch([{'host': 'localhost', 'port': 9200}])>
NoneType: None


KeyboardInterrupt: 